In [263]:
import pandas as pd
from sklearn.impute import SimpleImputer 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import VarianceThreshold

Membaca data

In [264]:
data = pd.read_csv("train.csv")

data = data.dropna()

In [265]:
# under_sampler = RandomUnderSampler(sampling_strategy={
#     'lived': 100,
#     'died': 100,
#     'euthanized': 100
# })

In [266]:
target = data[data.columns[-1]]
data = data.drop(columns=['id','hospital_number','outcome'])

In [267]:
# data,target = under_sampler.fit_resample(data,target)

In [268]:
data.shape

(771, 26)

In [269]:
data.dtypes

surgery                   object
age                       object
rectal_temp              float64
pulse                    float64
respiratory_rate         float64
temp_of_extremities       object
peripheral_pulse          object
mucous_membrane           object
capillary_refill_time     object
pain                      object
peristalsis               object
abdominal_distention      object
nasogastric_tube          object
nasogastric_reflux        object
nasogastric_reflux_ph    float64
rectal_exam_feces         object
abdomen                   object
packed_cell_volume       float64
total_protein            float64
abdomo_appearance         object
abdomo_protein           float64
surgical_lesion           object
lesion_1                   int64
lesion_2                   int64
lesion_3                   int64
cp_data                   object
dtype: object

In [270]:
data.head()

,surgery,age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,yes,adult,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,...,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no
1,yes,adult,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,...,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no
3,yes,adult,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,...,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes
4,no,adult,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,...,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes
5,no,adult,38.1,56.0,32.0,normal,normal,bright_pink,less_3_sec,depressed,...,firm,49.0,8.0,cloudy,2.8,no,0,0,0,yes


In [271]:
numerical_data = data.select_dtypes(include=[int, float])
categorical_data = data.select_dtypes(include=object)

In [272]:
imputer_numerical = SimpleImputer(strategy="constant", fill_value=0)
imputer_categorical = SimpleImputer(strategy="constant", fill_value="Tidak ada")

In [273]:
numerical_data_columns = numerical_data.columns
categorical_data_columns = categorical_data.columns

In [274]:
numerical_data.head()

,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
0,38.1,132.0,24.0,6.5,57.0,8.5,3.4,2209,0,0
1,37.5,88.0,12.0,2.0,33.0,64.0,2.0,2208,0,0
3,37.1,72.0,30.0,2.0,53.0,7.0,3.9,2208,0,0
4,38.0,52.0,48.0,7.0,47.0,7.3,2.6,0,0,0
5,38.1,56.0,32.0,5.0,49.0,8.0,2.8,0,0,0


In [275]:
categorical_data.head()

,surgery,age,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,surgical_lesion,cp_data
0,yes,adult,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,decreased,distend_small,serosanguious,yes,no
1,yes,adult,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,absent,distend_small,serosanguious,yes,no
3,yes,adult,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,decreased,distend_small,cloudy,yes,yes
4,no,adult,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,normal,normal,cloudy,no,yes
5,no,adult,normal,normal,bright_pink,less_3_sec,depressed,hypomotile,none,none,more_1_liter,decreased,firm,cloudy,no,yes


In [276]:
numerical_data = pd.DataFrame(imputer_numerical.fit(numerical_data).transform(numerical_data), columns=numerical_data_columns)
categorical_data = pd.DataFrame(imputer_categorical.fit(categorical_data).transform(categorical_data), columns=categorical_data_columns)

In [277]:
numerical_data = (numerical_data-numerical_data.mean())/numerical_data.std()

In [278]:
# numerical_data = (numerical_data-numerical_data.min())/(numerical_data.max()-numerical_data.min())

In [279]:
# for column in categorical_data_columns:
#     le = LabelEncoder()
#     le.fit(list(set(categorical_data[column])))
#     temp_list = categorical_data[column].to_list()
#     temp_list = le.transform(temp_list)
#     categorical_data[column] = pd.Series(temp_list)

In [280]:
le1 = LabelEncoder()
le1 = le1.fit(['Tidak ada','normal','warm','cool','cold'])
tmp = le1.transform(categorical_data['temp_of_extremities'].to_list())
categorical_data['temp_of_extremities'] = pd.Series(tmp)

In [281]:
le2 = LabelEncoder()
le2 = le2.fit(['Tidak ada','normal','increased','reduced','absent'])
tmp = le2.transform(categorical_data['peripheral_pulse'].to_list())
categorical_data['peripheral_pulse'] = pd.Series(tmp)

In [282]:
le3 = LabelEncoder()
le3 = le3.fit(['Tidak ada','normal_pink','bright_pink','pale_pink','pale_cyanotic','dark_cyanotic','bright_red'])
tmp = le3.transform(categorical_data['mucous_membrane'].to_list())
categorical_data['mucous_membrane'] = pd.Series(tmp)

In [283]:
le4 = LabelEncoder()
le4 = le4.fit(['Tidak ada','less_3_sec','3','more_3_sec'])
tmp = le4.transform(categorical_data['capillary_refill_time'].to_list())
categorical_data['capillary_refill_time'] = pd.Series(tmp)

In [284]:
le5 = LabelEncoder()
le5 = le5.fit(['Tidak ada','alert','slight','depressed','mild_pain','severe_pain','extreme_pain'])
tmp = le5.transform(categorical_data['pain'].to_list())
categorical_data['pain'] = pd.Series(tmp)

In [285]:
le6 = LabelEncoder()
le6 = le6.fit(['Tidak ada','distend_small','hypomotile','normal','hypermotile','absent'])
tmp = le6.transform(categorical_data['peristalsis'].to_list())
categorical_data['peristalsis'] = pd.Series(tmp)

In [286]:
print(set(categorical_data['abdominal_distention']))

{'slight', 'moderate', 'severe', 'none'}


In [287]:
le7 = LabelEncoder()
le7 = le7.fit(['Tidak ada','none','slight','moderate','severe'])
tmp = le7.transform(categorical_data['abdominal_distention'].to_list())
categorical_data['abdominal_distention'] = pd.Series(tmp)

In [288]:
print(set(categorical_data['nasogastric_tube']))

{'slight', 'none', 'significant'}


In [289]:
le8 = LabelEncoder()
le8 = le8.fit(['Tidak ada','none','slight','significant'])
tmp = le8.transform(categorical_data['nasogastric_tube'].to_list())
categorical_data['nasogastric_tube'] = pd.Series(tmp)

In [290]:
print(set(categorical_data['nasogastric_reflux']))

{'slight', 'none', 'less_1_liter', 'more_1_liter'}


In [291]:
le9 = LabelEncoder()
le9 = le9.fit(['Tidak ada','none','slight','less_1_liter','more_1_liter'])
tmp = le9.transform(categorical_data['nasogastric_reflux'].to_list())
categorical_data['nasogastric_reflux'] = pd.Series(tmp)

In [292]:
print(set(categorical_data['rectal_exam_feces']))

{'absent', 'decreased', 'serosanguious', 'increased', 'normal'}


In [293]:
le10 = LabelEncoder()
le10 = le10.fit(['Tidak ada','decreased','normal','increased','absent','serosanguious'])
tmp = le10.transform(categorical_data['rectal_exam_feces'].to_list())
categorical_data['rectal_exam_feces'] = pd.Series(tmp)

In [294]:
print(set(categorical_data['abdomen']))

{'firm', 'other', 'distend_large', 'distend_small', 'normal'}


In [295]:
le11 = LabelEncoder()
le11 = le11.fit(['Tidak ada','normal','other','firm','distend_small','distend_large'])
tmp = le11.transform(categorical_data['abdomen'].to_list())
categorical_data['abdomen'] = pd.Series(tmp)

In [296]:
print(set(categorical_data['abdomo_appearance']))

{'cloudy', 'serosanguious', 'clear'}


In [297]:
le12 = LabelEncoder()
le12 = le12.fit(['Tidak ada','clear','cloudy','serosanguious'])
tmp = le12.transform(categorical_data['abdomo_appearance'].to_list())
categorical_data['abdomo_appearance'] = pd.Series(tmp)

In [298]:
categorical_data.head()

,surgery,age,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,surgical_lesion,cp_data
0,yes,adult,2,4,3,3,2,1,4,3,1,2,2,3,yes,no
1,yes,adult,2,3,5,3,4,1,1,1,2,1,2,3,yes,no
2,yes,adult,1,4,6,3,4,4,1,3,2,2,2,2,yes,yes
3,no,adult,3,3,4,2,1,4,2,3,1,4,4,2,no,yes
4,no,adult,3,3,1,2,2,4,2,1,2,2,3,2,no,yes


In [299]:
le13 = LabelEncoder()
le13 = le13.fit(['Tidak ada','no','yes'])
tmp = le13.transform(categorical_data['surgery'].to_list())
categorical_data['surgery'] = pd.Series(tmp)

In [300]:
le14 = LabelEncoder()
le14 = le14.fit(['Tidak ada','no','yes'])
tmp = le14.transform(categorical_data['surgical_lesion'].to_list())
categorical_data['surgical_lesion'] = pd.Series(tmp)

In [301]:
le15 = LabelEncoder()
le15 = le14.fit(['Tidak ada','no','yes'])
tmp = le14.transform(categorical_data['cp_data'].to_list())
categorical_data['cp_data'] = pd.Series(tmp)

In [302]:
le16 = LabelEncoder()
le16 = le16.fit(['young','adult'])
tmp = le16.transform(categorical_data['age'].to_list())
categorical_data['age'] = pd.Series(tmp)

In [303]:
data = pd.merge(left=numerical_data, right=categorical_data, left_index=True, right_index=True, how='inner')
# data = numerical_data
# data = categorical_data

In [304]:
selector = VarianceThreshold(threshold=(.8 * (1 - .8)))

In [305]:
selector.fit_transform(data)

array([[-0.11230164,  2.01759245, -0.33608663, ...,  3.        ,
         2.        ,  1.        ],
       [-0.83990306,  0.38962094, -1.08015783, ...,  3.        ,
         2.        ,  1.        ],
       [-1.32497067, -0.2023687 ,  0.03594896, ...,  2.        ,
         2.        ,  2.        ],
       ...,
       [-0.83990306, -0.64636093,  1.27606762, ...,  3.        ,
         2.        ,  2.        ],
       [-0.83990306,  0.24162353,  0.65600829, ...,  2.        ,
         2.        ,  2.        ],
       [-0.11230164, -0.27636741, -0.83213409, ...,  2.        ,
         2.        ,  1.        ]])

In [307]:
selector.get_feature_names_out()

array(['rectal_temp', 'pulse', 'respiratory_rate',
       'nasogastric_reflux_ph', 'packed_cell_volume', 'total_protein',
       'abdomo_protein', 'lesion_1', 'lesion_2', 'lesion_3', 'surgery',
       'temp_of_extremities', 'peripheral_pulse', 'mucous_membrane',
       'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'rectal_exam_feces', 'abdomen', 'abdomo_appearance',
       'surgical_lesion', 'cp_data'], dtype=object)

In [ ]:
data.dtypes

rectal_temp              float64
pulse                    float64
respiratory_rate         float64
nasogastric_reflux_ph    float64
packed_cell_volume       float64
total_protein            float64
abdomo_protein           float64
lesion_1                 float64
lesion_2                 float64
lesion_3                 float64
surgery                    int32
age                        int32
temp_of_extremities        int32
peripheral_pulse           int32
mucous_membrane            int32
capillary_refill_time      int32
pain                       int32
peristalsis                int32
abdominal_distention       int32
nasogastric_tube           int32
nasogastric_reflux         int32
rectal_exam_feces          int32
abdomen                    int32
abdomo_appearance          int32
surgical_lesion            int32
cp_data                    int32
dtype: object

In [ ]:
data.head()

,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3,...,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,surgical_lesion,cp_data
0,-0.112302,2.017592,-0.336087,1.005897,0.740517,-0.371975,-0.032765,-0.314028,-0.062459,-0.050965,...,2,1,4,3,1,2,2,3,2,1
1,-0.839903,0.389621,-1.080158,-1.439251,-1.578604,1.985608,-0.958603,-0.314197,-0.062459,-0.050965,...,4,1,1,1,2,1,2,3,2,1
2,-1.324971,-0.202369,0.035949,-1.439251,0.353997,-0.435693,0.297891,-0.314197,-0.062459,-0.050965,...,4,4,1,3,2,2,2,2,2,2
3,-0.233569,-0.942356,1.152056,1.277580,-0.225783,-0.422950,-0.561815,-0.685578,-0.062459,-0.050965,...,1,4,2,3,1,4,4,2,1,2
4,-0.112302,-0.794358,0.159961,0.190848,-0.032523,-0.393214,-0.429553,-0.685578,-0.062459,-0.050965,...,2,4,2,1,2,2,3,2,1,2


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   rectal_temp            771 non-null    float64
 1   pulse                  771 non-null    float64
 2   respiratory_rate       771 non-null    float64
 3   nasogastric_reflux_ph  771 non-null    float64
 4   packed_cell_volume     771 non-null    float64
 5   total_protein          771 non-null    float64
 6   abdomo_protein         771 non-null    float64
 7   lesion_1               771 non-null    float64
 8   lesion_2               771 non-null    float64
 9   lesion_3               771 non-null    float64
 10  surgery                771 non-null    int32  
 11  age                    771 non-null    int32  
 12  temp_of_extremities    771 non-null    int32  
 13  peripheral_pulse       771 non-null    int32  
 14  mucous_membrane        771 non-null    int32  
 15  capill

In [ ]:
data.shape

(771, 26)

In [ ]:
model1 = KNeighborsClassifier(n_neighbors=3,weights='distance', algorithm='brute',p=2)
model2 = SVC()
model3 = DecisionTreeClassifier()
model4 = GaussianNB()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data,target, test_size=0.2)

In [ ]:
model1 = model1.fit(x_train, y_train)
model2 = model2.fit(x_train, y_train)
model3 = model3.fit(x_train, y_train)
model4 = model4.fit(x_train, y_train)

In [ ]:
temp1 = model1.predict(x_test).tolist()
temp2 = model2.predict(x_test).tolist()
temp3 = model3.predict(x_test).tolist()
temp4 = model4.predict(x_test).tolist()

In [ ]:
print(temp1)

['died', 'lived', 'lived', 'lived', 'euthanized', 'lived', 'lived', 'died', 'died', 'died', 'died', 'died', 'died', 'lived', 'died', 'lived', 'died', 'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'lived', 'died', 'died', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'euthanized', 'lived', 'died', 'died', 'died', 'lived', 'lived', 'euthanized', 'euthanized', 'died', 'lived', 'euthanized', 'lived', 'lived', 'died', 'lived', 'lived', 'euthanized', 'lived', 'euthanized', 'lived', 'euthanized', 'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'died', 'died', 'died', 'lived', 'died', 'died', 'died', 'died', 'died', 'lived', 'lived', 'euthanized', 'lived', 'died', 'lived', 'died', 'euthanized', 'lived', 'died', 'lived', 'died', 'died', 'euthanized', 'died', 'lived', 'lived', 'lived', 'died', 'lived', 'euthanized', 'died', 'lived', 'lived', 'died', 'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'died', 'died', 'died', 'lived', 'died', 'lived'

In [ ]:
print(temp2)

['died', 'lived', 'lived', 'lived', 'died', 'died', 'died', 'died', 'lived', 'lived', 'died', 'died', 'died', 'died', 'died', 'lived', 'lived', 'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'died', 'died', 'died', 'lived', 'lived', 'lived', 'died', 'died', 'died', 'lived', 'died', 'euthanized', 'euthanized', 'died', 'lived', 'lived', 'lived', 'lived', 'died', 'lived', 'lived', 'euthanized', 'lived', 'euthanized', 'lived', 'died', 'lived', 'died', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'died', 'died', 'died', 'lived', 'died', 'died', 'died', 'died', 'died', 'lived', 'lived', 'lived', 'died', 'died', 'lived', 'died', 'euthanized', 'died', 'died', 'lived', 'lived', 'died', 'euthanized', 'died', 'lived', 'lived', 'lived', 'died', 'lived', 'died', 'died', 'lived', 'lived', 'died', 'lived', 'died', 'died', 'died', 'died', 'euthanized', 'lived', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'lived', 'lived', 'd

In [ ]:
print(temp3)

['died', 'lived', 'lived', 'died', 'lived', 'lived', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'died', 'lived', 'euthanized', 'lived', 'lived', 'lived', 'died', 'lived', 'euthanized', 'lived', 'lived', 'lived', 'died', 'died', 'euthanized', 'lived', 'lived', 'euthanized', 'lived', 'died', 'died', 'lived', 'lived', 'died', 'lived', 'lived', 'lived', 'died', 'died', 'euthanized', 'lived', 'lived', 'died', 'lived', 'lived', 'euthanized', 'died', 'died', 'died', 'lived', 'lived', 'lived', 'died', 'lived', 'died', 'lived', 'died', 'died', 'euthanized', 'lived', 'lived', 'died', 'euthanized', 'lived', 'died', 'died', 'lived', 'lived', 'euthanized', 'lived', 'lived', 'lived', 'died', 'died', 'died', 'lived', 'died', 'euthanized', 'died', 'lived', 'died', 'lived', 'died', 'euthanized', 'died', 'lived', 'lived', 'euthanized', 'died', 'lived', 'died', 'euthanized', 'lived', 'lived', 'euthanized', 'lived', 'euthanized', 'died', 'died', 'euthanized', 'euthanized', 'lived', 'lived', 'di

In [ ]:
print(temp4)

['died', 'died', 'euthanized', 'died', 'died', 'euthanized', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'euthanized', 'died', 'died', 'euthanized', 'died', 'euthanized', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'euthanized', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'euthanized', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'euthanized', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'died', 'died', 'euthanized', 'died', 'died', 'euthanized', 'euthanized', 'died', 'died', 'euthanized', 'died', 'died', 'died', 'euthanized', 'euthanized', 'died', 'died', 'died', 'euthanized', 'euthaniz

In [ ]:
vote = []

for i in range (len(temp1)):
    combine = [temp1[i],temp2[i],temp3[i]]
    euthanized = combine.count("euthanized")
    lived = combine.count("lived")
    died = combine.count("died")

    if euthanized > 1:
        vote.append("euthanized")
    elif lived > 1:
        vote.append("lived")
    elif died > 1:
        vote.append("died")
    else :
        vote.append("lived")

In [ ]:
# print(vote)

In [ ]:
test_label = y_test.to_list()

In [ ]:
print(test_label)

['died', 'lived', 'euthanized', 'lived', 'died', 'euthanized', 'lived', 'died', 'euthanized', 'lived', 'lived', 'died', 'lived', 'died', 'euthanized', 'lived', 'euthanized', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'euthanized', 'died', 'lived', 'died', 'lived', 'died', 'died', 'died', 'lived', 'euthanized', 'lived', 'died', 'died', 'died', 'lived', 'died', 'died', 'euthanized', 'died', 'died', 'euthanized', 'lived', 'lived', 'died', 'died', 'died', 'euthanized', 'lived', 'lived', 'lived', 'died', 'lived', 'lived', 'lived', 'euthanized', 'lived', 'lived', 'lived', 'lived', 'died', 'died', 'lived', 'lived', 'died', 'died', 'died', 'died', 'lived', 'lived', 'lived', 'died', 'lived', 'died', 'lived', 'died', 'euthanized', 'lived', 'died', 'lived', 'lived', 'died', 'died', 'died', 'lived', 'lived', 'euthanized', 'died', 'lived', 'lived', 'lived', 'lived', 'lived', 'died', 'lived', 'euthanized', 'lived', 'lived', 'euthanized', 'euthanized', 'lived', 'died', 'died', 'eu

In [ ]:
akurasi = 0

for i in range (len(test_label)):
    if vote[i] == test_label[i]:
        akurasi = akurasi+1

print("akurasi : " + str(akurasi/len(test_label)))

akurasi : 0.6387096774193548


In [ ]:
akurasi = 0

for i in range (len(test_label)):
    if temp1[i] == test_label[i]:
        akurasi = akurasi+1

print("akurasi : " + str(akurasi/len(test_label)))

akurasi : 0.6193548387096774


In [ ]:
akurasi = 0

for i in range (len(test_label)):
    if temp2[i] == test_label[i]:
        akurasi = akurasi+1

print("akurasi : " + str(akurasi/len(test_label)))

akurasi : 0.6709677419354839


In [ ]:
akurasi = 0

for i in range (len(test_label)):
    if temp3[i] == test_label[i]:
        akurasi = akurasi+1

print("akurasi : " + str(akurasi/len(test_label)))

akurasi : 0.6129032258064516


In [ ]:
akurasi = 0

for i in range (len(test_label)):
    if temp4[i] == test_label[i]:
        akurasi = akurasi+1

print("akurasi : " + str(akurasi/len(test_label)))

akurasi : 0.432258064516129


In [ ]:
target.value_counts()

outcome
lived         383
died          281
euthanized    107
Name: count, dtype: int64

In [ ]:
data_test = pd.read_csv("test.csv")

data_test = data_test.drop(columns=['id','hho'])

KeyError: "['hho'] not found in axis"

In [ ]:
numerical_data = data_test.select_dtypes(include=[int, float])
categorical_data = data_test.select_dtypes(include=object)

data_test = pd.merge(left=numerical_data, right=categorical_data, left_index=True, right_index=True, how='inner')

In [ ]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   rectal_temp            824 non-null    float64
 1   pulse                  824 non-null    float64
 2   respiratory_rate       824 non-null    float64
 3   nasogastric_reflux_ph  824 non-null    float64
 4   packed_cell_volume     824 non-null    float64
 5   total_protein          824 non-null    float64
 6   abdomo_protein         824 non-null    float64
 7   surgery                824 non-null    object 
 8   peripheral_pulse       777 non-null    object 
 9   capillary_refill_time  818 non-null    object 
 10  pain                   795 non-null    object 
 11  peristalsis            805 non-null    object 
 12  abdominal_distention   802 non-null    object 
 13  rectal_exam_feces      699 non-null    object 
 14  abdomen                670 non-null    object 
 15  abdomo

In [ ]:
hasil = model3.predict(data_test)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- age
- lesion_1
- lesion_2
- lesion_3
- mucous_membrane
- ...
